In [1]:
from datetime import datetime

from binance.client import Client
from binance.enums import *

from coinmarketcapapi import CoinMarketCapAPI, CoinMarketCapAPIError

from datetime import datetime
from binance.helpers import round_step_size

import numpy as np
import pandas as pd
import json
import time
import sys


In [2]:
cmc = CoinMarketCapAPI('  ')

In [3]:
rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [4]:
rank_info

[{'id': 1,
  'name': 'Bitcoin',
  'symbol': 'BTC',
  'slug': 'bitcoin',
  'rank': 1,
  'is_active': 1,
  'first_historical_data': '2013-04-28T18:47:21.000Z',
  'last_historical_data': '2022-09-08T06:59:00.000Z',
  'platform': None},
 {'id': 1027,
  'name': 'Ethereum',
  'symbol': 'ETH',
  'slug': 'ethereum',
  'rank': 2,
  'is_active': 1,
  'first_historical_data': '2015-08-07T14:49:30.000Z',
  'last_historical_data': '2022-09-08T06:59:00.000Z',
  'platform': None},
 {'id': 825,
  'name': 'Tether',
  'symbol': 'USDT',
  'slug': 'tether',
  'rank': 3,
  'is_active': 1,
  'first_historical_data': '2015-02-25T13:34:26.000Z',
  'last_historical_data': '2022-09-08T06:59:00.000Z',
  'platform': {'id': 1027,
   'name': 'Ethereum',
   'symbol': 'ETH',
   'slug': 'ethereum',
   'token_address': '0xdac17f958d2ee523a2206206994597c13d831ec7'}},
 {'id': 3408,
  'name': 'USD Coin',
  'symbol': 'USDC',
  'slug': 'usd-coin',
  'rank': 4,
  'is_active': 1,
  'first_historical_data': '2018-10-08T18:49:2

In [5]:
rank_list = []

In [6]:
for rank in rank_info :
    rank_dictionary = {}
    rank_dictionary['symbol'] =  rank['symbol']
    rank_dictionary['rank'] =  rank['rank']
    rank_list.append(rank_dictionary)
    

In [7]:
rank_list

[{'symbol': 'BTC', 'rank': 1},
 {'symbol': 'ETH', 'rank': 2},
 {'symbol': 'USDT', 'rank': 3},
 {'symbol': 'USDC', 'rank': 4},
 {'symbol': 'BNB', 'rank': 5},
 {'symbol': 'BUSD', 'rank': 6},
 {'symbol': 'XRP', 'rank': 7},
 {'symbol': 'ADA', 'rank': 8},
 {'symbol': 'SOL', 'rank': 9},
 {'symbol': 'DOGE', 'rank': 10},
 {'symbol': 'DOT', 'rank': 11},
 {'symbol': 'MATIC', 'rank': 12},
 {'symbol': 'DAI', 'rank': 13},
 {'symbol': 'SHIB', 'rank': 14},
 {'symbol': 'TRX', 'rank': 15},
 {'symbol': 'AVAX', 'rank': 16},
 {'symbol': 'ETC', 'rank': 17},
 {'symbol': 'LEO', 'rank': 18},
 {'symbol': 'WBTC', 'rank': 19},
 {'symbol': 'UNI', 'rank': 20},
 {'symbol': 'LTC', 'rank': 21},
 {'symbol': 'ATOM', 'rank': 22},
 {'symbol': 'LINK', 'rank': 23},
 {'symbol': 'FTT', 'rank': 24},
 {'symbol': 'NEAR', 'rank': 25},
 {'symbol': 'LUNC', 'rank': 26},
 {'symbol': 'CRO', 'rank': 27},
 {'symbol': 'XMR', 'rank': 28},
 {'symbol': 'XLM', 'rank': 29},
 {'symbol': 'BCH', 'rank': 30},
 {'symbol': 'ALGO', 'rank': 31},
 {'

In [8]:
#client = Client('', '')

In [9]:
#cmc = CoinMarketCapAPI('  ')

In [10]:
#info = client.get_all_tickers()

In [11]:
#quote_ccy = 'USDT'

In [12]:
#ticker_list = [d['symbol'] for d in info if d['symbol'].endswith(quote_ccy)]

In [13]:
#rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data

In [14]:
#target_rank_info = [d['symbol']+quote_ccy for d in rank_info if d['rank'] >= 100 and d['rank'] <= 500]

In [15]:
#target_ticker_list = [d for d in ticker_list if d in target_rank_info]

In [16]:
#target_ticker_list

In [17]:
#target_ticker_list = ['BONDUSDT']

In [18]:
global client;

In [19]:
global target_ticker_list

In [20]:
def init():
    global client
    client = Client('', '', {"verify": True, "timeout": 20})
    cmc = CoinMarketCapAPI('  ')
    info_all_tickers  = client.get_all_tickers()
    quote_ccy = 'USDT'
    ticker_list = [d['symbol'] for d in info_all_tickers if d['symbol'].endswith(quote_ccy)]
    rank_info = cmc.cryptocurrency_map(sort='cmc_rank').data
    target_rank_info = [d['symbol']+quote_ccy for d in rank_info if d['rank'] >= 1 and d['rank'] <= 300]
    global target_ticker_list
    rank_filtered_ticker_list = [d for d in ticker_list if d in target_rank_info]
    # target_ticker_list = ['RIFUSDT']
    pair_details = client.get_all_isolated_margin_symbols()
    margin_pair_list = [d['symbol'] for d in pair_details if d['quote'] == quote_ccy and d['isMarginTrade'] == True]   
    target_ticker_list = [d for d in rank_filtered_ticker_list if d in margin_pair_list]
    #target_ticker_list = ['ARDRUSDT']
    #target_ticker_list = rank_filtered_ticker_list 
    return target_ticker_list
    

In [21]:
def is_candidate(filtered_klines_info): 
    is_candidate = False
    
    buy_kline_info = filtered_klines_info[0]
    buy_kline_average_volume = buy_kline_info[5]

    
    strong_context_kline_info = filtered_klines_info[1:len(filtered_klines_info)]


    
    strong_context_kline_average_volume = np.mean(strong_context_kline_info[:,5])
    
    for strong_context_kline in strong_context_kline_info:
        perc_change = strong_context_kline[-1]
        perc_volume = strong_context_kline[5] / buy_kline_average_volume


        if perc_change > 1 or perc_change < -2 or perc_volume > 0.5:
            return False
    

    buy_line_open_price = buy_kline_info[1]
    buy_line_high_price = buy_kline_info[2]
    buy_line_low_price = buy_kline_info[3]
    buy_line_close_price = buy_kline_info[4]
    

    #Drop from high price to close price should be in the range of 10 - 15%
    difference_high_price_vs_open_price = buy_line_high_price - buy_line_open_price
    difference_close_price_vs_open_price = buy_line_close_price - buy_line_open_price
    
    
    drop_from_high_to_close_gate =   1.9 > difference_high_price_vs_open_price/difference_close_price_vs_open_price > 1.85


    #Percentage increase should be in the range of 7 - 15%
    percentage_gate = 7 > buy_kline_info[-1] > 4

    if percentage_gate and True and True:

        is_candidate = True
    else:
        is_candidate = False

    return is_candidate

In [22]:
def find_percentage_increase_or_decrease(focused_kline_array):
    percentage_value = []
    for kline in focused_kline_array:
        diff = kline[4] - kline[1]
        perc_diff = diff/kline[1]
        percentage_value.append(perc_diff*100)
    return percentage_value 

In [23]:
trade_list = []
init()
test_run = False
print('Total Tickers to Analyze')
print(len(target_ticker_list))
count = 1
for target_ticker in target_ticker_list:
    
    print('Analyzing Hourly Started ')
    print(target_ticker)
    print(count)
    count = count + 1

    klines = client.get_historical_klines(target_ticker, Client.KLINE_INTERVAL_1HOUR, "2 day ago UTC") 
    

    
    
    
    length = int(len(klines))
    if(length < 24):
        continue
    

    #klines.pop()
    
    

    

    array = np.array(klines, dtype=np.float64)
    focused_kline_array = np.delete(array, [6, 7, 8, 9, 10, 11], 1)
    percentage_value = find_percentage_increase_or_decrease(focused_kline_array)
    # reduce lengrth below if poped
    percentage_value = np.reshape(percentage_value, (length, 1))
    focused_kline_array = np.append(focused_kline_array, percentage_value, 1)
    
    focused_reverse_kline_array = focused_kline_array[::-1]

    for x in range(0, length):

        hit_ticker_dictionary = {}
        
        filtered_klines_info = focused_reverse_kline_array[x:x+5]


        if(len(filtered_klines_info) == 5):


            
            if(is_candidate(filtered_klines_info)):
                    
                
                    
                buy_kline_info = filtered_klines_info[0]
                hit_ticker_dictionary = {}
                buy_price = buy_kline_info[4]
                hit_ticker_dictionary['ticker'] = target_ticker
                hit_ticker_dictionary['buy_price'] = buy_kline_info[4]

                buy_timesstamp = buy_kline_info[0]
                hit_ticker_dictionary['buy_datetime'] = str(datetime.fromtimestamp(buy_timesstamp // 1000))
                
                proft_percentage = 1.10
                loss_percentage = 0.8
                profit_sell_price = buy_price * proft_percentage
                loss_sell_price = buy_price * loss_percentage
                
                
                
                usdt = 3000
                fees_for_completed_trade = usdt * 0.15 / 100
                
                buy_quantity = usdt/buy_price
                hit_ticker_dictionary['quantity'] = buy_quantity
                
                
 
                
                for y in range(0, x) :
                    
                    start = x - (y + 1)
                    end = x - y
                    
                    high_price = focused_reverse_kline_array[start:end][0][2]
                    low_price = focused_reverse_kline_array[start:end][0][3]
                    close_price = focused_reverse_kline_array[start:end][0][4]
                    sell_timesstamp = focused_reverse_kline_array[start:end][0][0]
                    sell_datetime = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                    
                    if high_price >= profit_sell_price:
                        hit_ticker_dictionary['sell_price'] = profit_sell_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_reverse_kline_array[start:end][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = profit_sell_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = usdt * proft_percentage - usdt - fees_for_completed_trade
                        break
                    elif sell_timesstamp - buy_timesstamp >= 13200000 :
                        hit_ticker_dictionary['sell_price'] = close_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_reverse_kline_array[start:end][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = close_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = buy_quantity * close_price - buy_quantity * buy_price - fees_for_completed_trade
                        break   
                    elif False: # low_price <=  loss_sell_price:
                        hit_ticker_dictionary['sell_price'] = loss_sell_price

                        hit_ticker_dictionary['sell_datetime'] = str(datetime.fromtimestamp(sell_timesstamp // 1000))
                        difference = focused_reverse_kline_array[start:end][0][0] - buy_kline_info[0]
                        
                        hit_ticker_dictionary['days_to_gain_loss'] = round(difference/(1000 * 60 * 24 * 60))
                        price_difference = loss_sell_price - buy_price
                        hit_ticker_dictionary['price_diff'] = price_difference
                        hit_ticker_dictionary['proft_loss'] = usdt * loss_percentage - usdt - fees_for_completed_trade
                        break
                        
                        
                trade_list.append(hit_ticker_dictionary)        


Total Tickers to Analyze
177
Analyzing Hourly Started 
BTCUSDT
1
Analyzing Hourly Started 
ETHUSDT
2
Analyzing Hourly Started 
BNBUSDT
3
Analyzing Hourly Started 
NEOUSDT
4
Analyzing Hourly Started 
LTCUSDT
5
Analyzing Hourly Started 
QTUMUSDT
6
Analyzing Hourly Started 
ADAUSDT
7
Analyzing Hourly Started 
XRPUSDT
8
Analyzing Hourly Started 
EOSUSDT
9
Analyzing Hourly Started 
TUSDUSDT
10
Analyzing Hourly Started 
XLMUSDT
11
Analyzing Hourly Started 
ONTUSDT
12
Analyzing Hourly Started 
TRXUSDT
13
Analyzing Hourly Started 
ETCUSDT
14
Analyzing Hourly Started 
ICXUSDT
15
Analyzing Hourly Started 
VETUSDT
16
Analyzing Hourly Started 
USDCUSDT
17


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in double_scalars
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in double_scalars


Analyzing Hourly Started 
LINKUSDT
18
Analyzing Hourly Started 
WAVESUSDT
19
Analyzing Hourly Started 
ONGUSDT
20


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()


Analyzing Hourly Started 
HOTUSDT
21
Analyzing Hourly Started 
ZILUSDT
22
Analyzing Hourly Started 
ZRXUSDT
23
Analyzing Hourly Started 
BATUSDT
24
Analyzing Hourly Started 
XMRUSDT
25
Analyzing Hourly Started 
ZECUSDT
26
Analyzing Hourly Started 
IOSTUSDT
27
Analyzing Hourly Started 
CELRUSDT
28
Analyzing Hourly Started 
DASHUSDT
29
Analyzing Hourly Started 
OMGUSDT
30
Analyzing Hourly Started 
THETAUSDT
31
Analyzing Hourly Started 
ENJUSDT
32
Analyzing Hourly Started 
MATICUSDT
33
Analyzing Hourly Started 
ATOMUSDT
34
Analyzing Hourly Started 
TFUELUSDT
35
Analyzing Hourly Started 
ONEUSDT
36
Analyzing Hourly Started 
FTMUSDT
37
Analyzing Hourly Started 
ALGOUSDT
38
Analyzing Hourly Started 
DOGEUSDT
39
Analyzing Hourly Started 
ANKRUSDT
40
Analyzing Hourly Started 
WINUSDT
41
Analyzing Hourly Started 
MTLUSDT
42
Analyzing Hourly Started 
DENTUSDT
43
Analyzing Hourly Started 
FUNUSDT
44
Analyzing Hourly Started 
CVCUSDT
45
Analyzing Hourly Started 
CHZUSDT
46
Analyzing Hourly Started

In [24]:
trade_list

[{'ticker': 'ETHUSDT',
  'buy_price': 1639.73,
  'buy_datetime': '2022-09-07 22:00:00',
  'quantity': 1.8295695022961096,
  'sell_price': 1628.03,
  'sell_datetime': '2022-09-08 02:00:00',
  'days_to_gain_loss': 0,
  'price_diff': -11.700000000000045,
  'proft_loss': -25.90596317686459},
 {'ticker': 'ETCUSDT',
  'buy_price': 35.66,
  'buy_datetime': '2022-09-07 15:00:00',
  'quantity': 84.12787436904095,
  'sell_price': 36.11,
  'sell_datetime': '2022-09-07 19:00:00',
  'days_to_gain_loss': 0,
  'price_diff': 0.45000000000000284,
  'proft_loss': 33.35754346606882},
 {'ticker': 'ATOMUSDT',
  'buy_price': 12.688,
  'buy_datetime': '2022-09-07 22:00:00',
  'quantity': 236.44388398486757,
  'sell_price': 12.833,
  'sell_datetime': '2022-09-08 02:00:00',
  'days_to_gain_loss': 0,
  'price_diff': 0.14499999999999957,
  'proft_loss': 29.784363177805517},
 {'ticker': 'TFUELUSDT',
  'buy_price': 0.0554,
  'buy_datetime': '2022-09-07 03:00:00',
  'quantity': 54151.624548736465,
  'sell_price': 0

In [25]:
a = 10
b = 5

step = -1 if a > b else 1
for x in range (a, b, step):
    print(x, end = ' ')

10 9 8 7 6 

In [26]:
len(trade_list)

5

In [27]:
df = pd.DataFrame.from_dict(trade_list)

In [28]:
print('Profit and Loss')
df['proft_loss'].sum()

Profit and Loss


53.75047976804535

In [29]:
print('Number of Profit Trades')
df[df['price_diff'] > 0].shape[0]

Number of Profit Trades


3

In [30]:
print('Number of Loss Trades')
df[df['price_diff'] < 0].shape[0]

Number of Loss Trades


2

In [31]:
with pd.option_context('display.max_rows', None):
    display(df[np.invert(pd.isna(df['sell_price']))].sort_values('buy_datetime'))

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss
4,GALUSDT,2.6360,2022-09-06 15:00:00,1138.088012,2.7060,2022-09-06 19:00:00,0,0.070,75.166161
3,TFUELUSDT,0.0554,2022-09-07 03:00:00,54151.624549,0.0544,2022-09-07 07:00:00,0,-0.001,-58.651625
1,ETCUSDT,35.6600,2022-09-07 15:00:00,84.127874,36.1100,2022-09-07 19:00:00,0,0.450,33.357543
0,ETHUSDT,1639.7300,2022-09-07 22:00:00,1.829570,1628.0300,2022-09-08 02:00:00,0,-11.700,-25.905963
2,ATOMUSDT,12.6880,2022-09-07 22:00:00,236.443884,12.8330,2022-09-08 02:00:00,0,0.145,29.784363


In [32]:
with pd.option_context('display.max_rows', None):
    display(df.sort_values('buy_datetime'))

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss
4,GALUSDT,2.6360,2022-09-06 15:00:00,1138.088012,2.7060,2022-09-06 19:00:00,0,0.070,75.166161
3,TFUELUSDT,0.0554,2022-09-07 03:00:00,54151.624549,0.0544,2022-09-07 07:00:00,0,-0.001,-58.651625
1,ETCUSDT,35.6600,2022-09-07 15:00:00,84.127874,36.1100,2022-09-07 19:00:00,0,0.450,33.357543
0,ETHUSDT,1639.7300,2022-09-07 22:00:00,1.829570,1628.0300,2022-09-08 02:00:00,0,-11.700,-25.905963
2,ATOMUSDT,12.6880,2022-09-07 22:00:00,236.443884,12.8330,2022-09-08 02:00:00,0,0.145,29.784363


In [33]:
with pd.option_context('display.max_rows', None):
    display(df[df['price_diff'] > 0].sort_values('buy_datetime'))

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss
4,GALUSDT,2.636,2022-09-06 15:00:00,1138.088012,2.706,2022-09-06 19:00:00,0,0.070,75.166161
1,ETCUSDT,35.660,2022-09-07 15:00:00,84.127874,36.110,2022-09-07 19:00:00,0,0.450,33.357543
2,ATOMUSDT,12.688,2022-09-07 22:00:00,236.443884,12.833,2022-09-08 02:00:00,0,0.145,29.784363


In [34]:
with pd.option_context('display.max_rows', None):
    display(df[df['price_diff'] < 0].sort_values('buy_datetime'))

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss
3,TFUELUSDT,0.0554,2022-09-07 03:00:00,54151.624549,0.0544,2022-09-07 07:00:00,0,-0.001,-58.651625
0,ETHUSDT,1639.7300,2022-09-07 22:00:00,1.829570,1628.0300,2022-09-08 02:00:00,0,-11.700,-25.905963


In [35]:
with pd.option_context('display.max_rows', None):
    display(df[pd.isna(df['sell_price'])].sort_values('buy_datetime'))

,ticker,buy_price,buy_datetime,quantity,sell_price,sell_datetime,days_to_gain_loss,price_diff,proft_loss


In [36]:
info = client.get_symbol_info('FLMUSDT')
print(info['filters'])

[{'filterType': 'PRICE_FILTER', 'minPrice': '0.00010000', 'maxPrice': '1000.00000000', 'tickSize': '0.00010000'}, {'filterType': 'PERCENT_PRICE', 'multiplierUp': '5', 'multiplierDown': '0.2', 'avgPriceMins': 5}, {'filterType': 'LOT_SIZE', 'minQty': '1.00000000', 'maxQty': '900000.00000000', 'stepSize': '1.00000000'}, {'filterType': 'MIN_NOTIONAL', 'minNotional': '10.00000000', 'applyToMarket': True, 'avgPriceMins': 5}, {'filterType': 'ICEBERG_PARTS', 'limit': 10}, {'filterType': 'MARKET_LOT_SIZE', 'minQty': '0.00000000', 'maxQty': '1171466.06875000', 'stepSize': '0.00000000'}, {'filterType': 'TRAILING_DELTA', 'minTrailingAboveDelta': 10, 'maxTrailingAboveDelta': 2000, 'minTrailingBelowDelta': 10, 'maxTrailingBelowDelta': 2000}, {'filterType': 'MAX_NUM_ORDERS', 'maxNumOrders': 200}, {'filterType': 'MAX_NUM_ALGO_ORDERS', 'maxNumAlgoOrders': 5}]


In [37]:
print(info['filters'][0]['tickSize'])

0.00010000


In [38]:
symbol_info = {}
def get_symbol_info(symbols):
    global symbol_info
    for symbol in symbols:
        info = client.get_symbol_info(symbol)
        base_asset = info['baseAsset']
#  PRICE_FILTER
        price_filter_min_quantity = float(info['filters'][0]['minPrice'])
        price_filter_max_quantity = float(info['filters'][0]['maxPrice'])
        price_filter_tick_size = float(info['filters'][0]['tickSize'])
        
#  LOT_SIZE (for market- and limit orders)
        lot_size_min_quantity = float(info['filters'][2]['minQty'])
        lot_size_max_quantity = float(info['filters'][2]['maxQty'])
        lot_size_step_size = float(info['filters'][2]['stepSize'])
        
#  MIN NOTIONAL
        min_notional = float(info['filters'][3]['minNotional'])
        symbol_info[symbol] = {'price_filter_min_quantity': price_filter_min_quantity, 'price_filter_max_quantity': price_filter_max_quantity,\
                               'price_filter_tick_size': price_filter_tick_size,'lot_size_min_quantity': lot_size_min_quantity, \
                                'lot_size_min_quantity': lot_size_min_quantity, 'lot_size_max_quantity': lot_size_max_quantity, \
                               'lot_size_step_size': lot_size_step_size, 'min_notional': min_notional, }
        


In [39]:
info = client.get_margin_asset(asset='BNB')

In [40]:
info

{'assetName': 'BNB',
 'assetFullName': 'Binance Coin',
 'isBorrowable': True,
 'isMortgageable': True,
 'userMinBorrow': '0',
 'userMinRepay': '0'}

In [41]:
info = client.get_margin_symbol(symbol='BURGERUSDT')

In [42]:
pair_details = client.get_all_isolated_margin_symbols()

In [43]:
info = client.get_isolated_margin_symbol(symbol='BURGERUSDT')

In [44]:
info = client.get_all_isolated_margin_symbols()

In [45]:
info = client.get_margin_price_index(symbol='MOVRUSDT')

In [46]:
info

{'symbol': 'MOVRUSDT', 'price': '12.16507708', 'calcTime': 1662620895000}

In [47]:
#transaction = client.transfer_spot_to_margin(asset='USDT', amount='100')

In [48]:
#transaction = client.create_margin_loan(asset='USDT', amount='200')

In [49]:
#transaction

In [50]:
#details = client.get_margin_loan_details(asset='USDT', txId='114199991053')

In [51]:
#transaction = client.repay_margin_loan(asset='USDT', amount='200')

In [52]:
#transaction

In [53]:
#details = client.get_margin_loan_details(asset='USDT', txId='114199991053')

In [54]:
#details

In [55]:
#order = client.create_margin_order(symbol='MOVRUSDT', side=SIDE_SELL, type='MARKET', quoteOrderQty=200, isIsolated='TRUE')

In [56]:
#transaction = client.transfer_spot_to_isolated_margin(asset='USDT', symbol='MOVRUSDT', amount='100')

In [57]:
#transaction

In [58]:
#transaction = client.create_margin_loan(asset='USDT', amount='200', symbol='MOVRUSDT', isIsolated='TRUE')

In [59]:
#transaction = client.repay_margin_loan(asset='USDT', amount='200', symbol='MOVRUSDT', isIsolated='TRUE')

In [60]:
#transaction = client.create_margin_loan(asset='MOVR', amount='200', symbol='MOVRUSDT', isIsolated='TRUE')

In [61]:
#transaction = client.transfer_isolated_margin_to_spot(asset='USDT', symbol='MOVRUSDT', amount='99.99833334')

In [62]:
#details = client.get_max_margin_transfer

In [63]:
#details

In [64]:
orders = client.get_all_margin_orders(symbol='BURGERUSDT', limit=10)

In [65]:
orders

[]

In [66]:
info = client.get_margin_account()

In [67]:
info

{'tradeEnabled': True,
 'transferEnabled': True,
 'borrowEnabled': True,
 'marginLevel': '999',
 'totalAssetOfBtc': '0.00520938',
 'totalLiabilityOfBtc': '0.00000007',
 'totalNetAssetOfBtc': '0.00520931',
 'userAssets': [{'asset': 'AGLD',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': '0',
   'netAsset': '0'},
  {'asset': 'MATIC',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': '0',
   'netAsset': '0'},
  {'asset': 'BURGER',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': '0',
   'netAsset': '0'},
  {'asset': 'STPT',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': '0',
   'netAsset': '0'},
  {'asset': 'CTXC',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': '0',
   'netAsset': '0'},
  {'asset': 'STG',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': '0',
   'netAsset': '0'},
  {'asset': 'IOTX',
   'free': '0',
   'locked': '0',
   'borrowed': '0',
   'interest': 

In [68]:
#transaction = client.transfer_margin_to_spot(asset='BTC', amount='0.00430996')

In [69]:
transaction

NameError: name 'transaction' is not defined

In [ ]:
def adjust_price(target_ticker, price):
    info = client.get_symbol_info(target_ticker)
    tick_size = info['filters'][0]['tickSize']
    adjusted_price = round_step_size(price, tick_size)
    adjusted_price = np.format_float_positional(adjusted_price, trim='-')
    return adjusted_price

In [ ]:
order = client.get_order(symbol='SHIBUSDT', orderId=1554410320)

In [ ]:
order

In [ ]:
buy_price = 44.75
sell_price = float(buy_price) * 1.10

In [ ]:
sell_price

In [ ]:
adjusted_sell_price = adjust_price('ETCUSDT', sell_price)

In [ ]:
adjusted_sell_price

In [ ]:
'{0:f}'.format(sell_price)

In [ ]:
info = client.get_symbol_info('SHIBUSDT')


In [ ]:
quoteAssetPrecision = info['quoteAssetPrecision']

In [ ]:
'{0:f}'.format(sell_price)

In [ ]:
'{:.20f}'.format(a)

In [ ]:
f'{sell_price:.}'

In [ ]:
np.format_float_positional(buy_price, trim='-')

In [ ]:
cmc = CoinMarketCapAPI('  ')